# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np
import requests
import gmaps
import pprint
import time
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#read csv file
cities_wether_path = "final_cities.csv"
cities_main_data = pd.read_csv(cities_wether_path)
cities_main_data_df = pd.DataFrame(cities_main_data)
cities_main_data_df.head()

,City,Latitudes,Longitudes,Temperature,Humidity,Cloudiness,Wind_Speed,Country,Date
0,Ushuaia,-54.8000,-68.3000,9.00,76,75,5.66,AR,1612669624
1,Rabo de Peixe,37.8000,-25.5833,15.00,94,75,7.72,PT,1612669816
2,Bulgan,48.8125,103.5347,-10.89,87,0,1.06,MN,1612669817
3,Busselton,-33.6500,115.3333,16.11,94,100,0.89,AU,1612669623
4,Sittwe,20.1500,92.9000,23.64,38,0,2.99,MM,1612669818


In [3]:
cities_main_data_df.count()

City           718
Latitudes      718
Longitudes     718
Temperature    718
Humidity       718
Cloudiness     718
Wind_Speed     718
Country        711
Date           718
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Configure gmaps.
gmaps.configure(api_key=g_key)

In [5]:
#Use the Lat and Lng as locations and Humidity as the weight.
locations = cities_main_data_df[["Latitudes","Longitudes"]].astype(float)

locations = locations.dropna()
hum_weight = cities_main_data_df["Humidity"].astype(float)

fig = gmaps.figure()
hum_heat_layer = gmaps.heatmap_layer(locations, weights=hum_weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
# Add Heatmap layer to map.
fig.add_layer(hum_heat_layer)

Heatmap = fig
Heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Filter rows by Multiple Conditions
# -  Max temperature > 80 degrees but >70 (70°F ~ 21.11°C & 80°F ~26.67°C)
# -  Wind speed < 10 mph.
# -  Zero cloudiness.

Ideal_wether1 = cities_main_data_df[(cities_main_data_df['Temperature'] >21.11) & (cities_main_data_df['Temperature'] < 26.67)]
Ideal_wether2 = Ideal_wether1[(Ideal_wether1['Wind_Speed']<10)]
Ideal_wether_final = Ideal_wether2[(Ideal_wether1['Cloudiness']==0)]
Ideal_wether_final.head()

<ipython-input-6-8772997587fb>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Ideal_wether_final = Ideal_wether2[(Ideal_wether1['Cloudiness']==0)]


,City,Latitudes,Longitudes,Temperature,Humidity,Cloudiness,Wind_Speed,Country,Date
4,Sittwe,20.1500,92.9000,23.64,38,0,2.99,MM,1612669818
47,Vao,-22.6667,167.4833,25.86,80,0,5.09,NC,1612669545
111,Marsh Harbour,26.5412,-77.0636,23.81,89,0,7.91,BS,1612669893
167,Puri,19.8000,85.8500,22.00,49,0,3.09,IN,1612669930
183,Betioky,-23.7167,44.3833,25.32,79,0,1.05,MG,1612669941


In [7]:
Ideal_wether_final.count()

City           23
Latitudes      23
Longitudes     23
Temperature    23
Humidity       23
Cloudiness     23
Wind_Speed     23
Country        23
Date           23
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Ideal_wether_final["Hotel Name"] = ""
# Ideal_wether_final.head()

#check city search
city_locations = Ideal_wether_final[['Latitudes', 'Longitudes']]
city_locations.head()
# fig = gmaps.figure()
# markers = gmaps.marker_layer(city_locations)
# fig.add_layer(markers)
# fig

,Latitudes,Longitudes
4,20.1500,92.9000
47,-22.6667,167.4833
111,26.5412,-77.0636
167,19.8000,85.8500
183,-23.7167,44.3833


In [9]:
# Add columns for lat, lng, airport name, airport address, airport rating
# Note that we used "" to specify initial entry.
Ideal_wether_final1=Ideal_wether_final.copy()
Ideal_wether_final1["Hotel Name"] = ""
Ideal_wether_final1["Hotel Address"] = ""
Ideal_wether_final1["Hotel Rating"] = ""
Ideal_wether_final1.head()

,City,Latitudes,Longitudes,Temperature,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel Name,Hotel Address,Hotel Rating
4,Sittwe,20.1500,92.9000,23.64,38,0,2.99,MM,1612669818,,,
47,Vao,-22.6667,167.4833,25.86,80,0,5.09,NC,1612669545,,,
111,Marsh Harbour,26.5412,-77.0636,23.81,89,0,7.91,BS,1612669893,,,
167,Puri,19.8000,85.8500,22.00,49,0,3.09,IN,1612669930,,,
183,Betioky,-23.7167,44.3833,25.32,79,0,1.05,MG,1612669941,,,


In [10]:
# params to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng  to identify hotel closeby
for index, row in Ideal_wether_final1.iterrows():
    # get lat, lng from df
    lat = row["Latitudes"]
    lng = row["Longitudes"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"


    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
#     print(json.dumps(name_address, indent=4, sort_keys=True))

    #  try-except to skip any  missing a data point.
    try:
        Ideal_wether_final1.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        Ideal_wether_final1.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
        Ideal_wether_final1.loc[index, "Hotel Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [11]:
# Save Data to csv
Ideal_wether_final1.to_csv("Hotel_Output.csv")

# Visualize to confirm airport data appears
Ideal_wether_final1.head(10)


,City,Latitudes,Longitudes,Temperature,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel Name,Hotel Address,Hotel Rating
4,Sittwe,20.1500,92.9000,23.64,38,0,2.99,MM,1612669818,Hotel Memory,"19, Akaut Yone Street, Kyae Oin Gyi Quarter, S...",3.8
47,Vao,-22.6667,167.4833,25.86,80,0,5.09,NC,1612669545,OURE TERA beach resort,Kuto,4.3
111,Marsh Harbour,26.5412,-77.0636,23.81,89,0,7.91,BS,1612669893,Abaco Beach Resort,"Bay St, Marsh Harbour",4.2
167,Puri,19.8000,85.8500,22.00,49,0,3.09,IN,1612669930,Hotel Golden Palace Puri,"On Blue Flag Golden Beach of Puri, V.I.P Road,...",4.2
183,Betioky,-23.7167,44.3833,25.32,79,0,1.05,MG,1612669941,Betioky,"Betioky, Madagascar",
188,Imbituba,-28.2400,-48.6703,22.00,73,0,2.06,BR,1612669944,Praia Hotel Imbituba,mts do Porto de Imbituba - R. Manoel Antônio d...,4.6
194,Jati,24.3539,68.2671,21.64,22,0,3.16,PK,1612669949,National Bank of Pakistan,Jati,2
209,Acajutla,13.5928,-89.8275,24.55,74,0,2.19,SV,1612669960,Villavela Hotel Boutique,"Avenida Miramar, Acajutla",4.5
212,Jieshi,22.8134,115.8257,22.22,62,0,6.85,CN,1612669962,维也纳酒店,陆丰碣石镇玄武路4号,0
216,Pindorama,-21.1858,-48.9072,23.00,49,0,1.54,BR,1612669965,PINDORAMA GOLDEN PLAZA HOTEL,"Av. João de Souza Sobrinho, 1790 - Jardim Prim...",4.3


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in Ideal_wether_final1.iterrows()]
locations = Ideal_wether_final[["Latitudes", "Longitudes"]]


In [13]:
#hotel markers only
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
# fig

In [14]:
#Use the Lat and Lng as locations and Humidity as the weight.
locations = cities_main_data_df[["Latitudes","Longitudes"]].astype(float)

locations = locations.dropna()
hum_weight = cities_main_data_df["Humidity"].astype(float)

fig = gmaps.figure()
hum_heat_layer = gmaps.heatmap_layer(locations, weights=hum_weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)


# Add  layer to map.
fig.add_layer(hum_heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))